In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

import os, shutil
import torch.nn.functional as F

import torch.optim as optim
from torchvision import datasets, transforms, utils

import numpy as np
import matplotlib.pyplot as plt

import torchvision
from torchvision.models import vgg16

from itertools import islice


In [2]:
original_dataset_dir = 'C:/Users/pauli/Documents/M2/ML et DL/projet/Git/DamageDetection'

In [3]:
train_damage_dir = original_dataset_dir + '/train_another/damage'
validation_damage_dir = original_dataset_dir + '/validation_another/damage'
test_damage_dir = original_dataset_dir + '/test/damage'

train_nodamage_dir = original_dataset_dir + '/train_another/no_damage'
validation_nodamage_dir = original_dataset_dir + '/validation_another/no_damage'
test_nodamage_dir = original_dataset_dir + '/test/no_damage'

In [4]:
print('total training damage images: ',len(os.listdir(train_damage_dir)))
print('total validation damage images: ',len(os.listdir(validation_damage_dir)))
print('total test damage images: ',len(os.listdir(test_damage_dir)))

print('total training no damage images: ',len(os.listdir(train_nodamage_dir)))
print('total validation no damage images: ',len(os.listdir(validation_nodamage_dir)))
print('total test no damage images: ',len(os.listdir(test_nodamage_dir)))

total training damage images:  5000
total validation damage images:  1000
total test damage images:  1000
total training no damage images:  5000
total validation no damage images:  1000
total test no damage images:  1000


In [5]:
   
    
    
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, (3,3))
        self.pool1 = nn.MaxPool2d((2,2))
        self.conv2 = nn.Conv2d(32, 64, (3,3))
        self.pool2 = nn.MaxPool2d((2,2))
        self.conv3 = nn.Conv2d(64, 128, (3,3))
        self.pool3 = nn.MaxPool2d((2,2))
        self.conv4 = nn.Conv2d(128, 128, (3,3))
        self.pool4 = nn.MaxPool2d((2,2))
        self.fc1 = nn.Linear(128 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        x = self.pool2(x)
        x = F.relu(self.conv4(x))
        x = self.pool3(x)
        x = x.view(-1, 128 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x
    
    

In [6]:
model = CNN()

In [7]:
import torchsummary
model = CNN()
torchsummary.summary(model, (3,150,150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 148, 148]             896
         MaxPool2d-2           [-1, 32, 74, 74]               0
            Conv2d-3           [-1, 64, 72, 72]          18,496
         MaxPool2d-4           [-1, 64, 36, 36]               0
            Conv2d-5          [-1, 128, 34, 34]          73,856
         MaxPool2d-6          [-1, 128, 17, 17]               0
            Conv2d-7          [-1, 128, 15, 15]         147,584
         MaxPool2d-8            [-1, 128, 7, 7]               0
            Linear-9                  [-1, 512]       3,211,776
           Linear-10                    [-1, 1]             513
Total params: 3,453,121
Trainable params: 3,453,121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.26
Forward/backward pass size (MB): 11.53
Params size (MB): 13.17
Estim

In [8]:
#criterion = nn.BCELoss() #torch.nn.functional.binary_cross_entropy()
#optimizer = optim.RMSprop(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)

In [9]:
train_datagen = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    #transforms.Normalize((0,), (1/255,))
])

train_dataset = ImageFolder('C:/Users/pauli/Documents/M2/ML et DL/projet/Git/DamageDetection/train_another', transform=train_datagen)
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

validation_datagen = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    #transforms.Normalize((0,), (1/255,))
])

val_dataset = ImageFolder('C:/Users/pauli/Documents/M2/ML et DL/projet/Git/DamageDetection/validation_another', transform=validation_datagen)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)


In [10]:
print('Found {} images belonging to {} classes.'.format(len(train_dataset), len(train_dataset.classes)))
print('Found {} images belonging to {} classes.'.format(len(val_dataset), len(val_dataset.classes)))

Found 10000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [11]:
def train_generator():
    while True:
        # Générer des données d'entraînement
        x_train = torch.randn(100, 10)
        y_train = torch.randn(100, 10)
        yield x_train, y_train
        
def validation_generator():
    while True:
        # Générer des données de validation
        x_val = torch.randn(50, 10)
        y_val = torch.randn(50, 10)
        yield x_val, y_val

In [15]:
x_train, y_train = next(train_generator())
y_train

tensor([[ 6.4082e-01, -1.1519e+00, -4.1255e-01,  1.0767e+00,  6.1783e-01,
         -3.6488e-01, -9.3634e-02,  1.2111e+00, -2.2659e+00, -6.3870e-01],
        [ 1.4201e+00, -3.4495e-01,  1.9922e+00, -1.0202e+00, -3.1095e-01,
          1.2710e+00,  1.8856e+00, -1.5007e+00,  7.5116e-01, -1.2805e-01],
        [ 4.2689e-01, -5.3481e-01, -3.9927e-02, -1.7430e+00, -2.0821e-01,
          4.0943e-01, -4.2661e-01,  9.6910e-01, -2.7786e-01, -1.5069e+00],
        [ 1.1834e+00,  2.3697e+00,  2.4259e-01, -5.2080e-01, -5.8508e-01,
          2.1420e+00, -1.2929e+00,  7.4662e-01,  1.5386e-01, -8.0978e-01],
        [ 3.9163e-01,  1.7918e+00,  1.4180e+00,  3.2978e-01,  9.4741e-01,
         -4.5704e-01, -1.5261e+00,  2.4560e+00,  7.5325e-01,  4.4360e-01],
        [-1.8348e+00, -4.5262e-01, -7.0818e-01, -6.1026e-01,  6.7193e-01,
          2.6044e-01,  1.5610e+00, -5.3570e-01, -5.1263e-02,  6.5469e-01],
        [-1.0760e+00, -7.5016e-01,  2.0218e+00, -5.1871e-01,  9.2358e-02,
         -1.6072e+00,  4.7429e-0

In [30]:

criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)

# Initialisation des paramètres d'entraînement
steps_per_epoch = 100
epochs = 50
validation_steps = 50

loss_values = []

def fit(model, train_loader, val_loader, epochs, steps_per_epoch, validation_steps):
    for epoch in range(epochs):
        accuracy = 0.0
        running_loss = 0.0
        correct = 0 
        total = 0
        # Entraîne le modèle sur les données d'entraînement
        for i, (images, labels) in enumerate(train_loader):
            # Entraîne le modèle sur un lot de données
            outputs = model(images)
            loss = criterion(outputs, labels.float().view(-1,1))
            loss_values.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #running_loss += loss.item()
            accuracy += (torch.round(outputs) == labels.float().view(-1,1)).sum().item() / len(labels)
            running_loss += loss.item()
        
            # Affiche la progression de l'entraînement
            if i % 100 == 99 :
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{steps_per_epoch}], Loss: {running_loss / (i + 1)}, Acc : {accuracy / (i + 1)}")
                break
        train_loss = running_loss / steps_per_epoch
        train_acc = accuracy / steps_per_epoch * 100
                
        

        # Valide le modèle sur les données de validation
        with torch.no_grad():
            val_loss = 0
            correct = 0
            total = 0
            for i, (images, labels) in enumerate(val_loader):
                outputs = model(images)
                val_loss += criterion(outputs, labels.float().view(-1,1)).item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                if i >= validation_steps:
                    break
        val_loss /= validation_steps
        val_acc = correct / total * 100

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {loss.item()}, Train Accuracy: {train_acc:.2f}%, Validation Loss: {val_loss:.2f}, Validation Accuracy: {val_acc:.2f}%")

# Lancer l'entraînement
fit(model, train_loader, val_loader, epochs, steps_per_epoch, validation_steps)



Epoch [1/50], Step [100/100], Loss: 0.19825626148842276, Acc : 0.9460000000000008
Epoch [1/50], Train Loss: 0.01199197955429554, Train Accuracy: 94.60%, Validation Loss: 0.10, Validation Accuracy: 98.04%
Epoch [2/50], Step [100/100], Loss: 0.09887573370710015, Acc : 0.9600000000000009
Epoch [2/50], Train Loss: 0.11488822847604752, Train Accuracy: 96.00%, Validation Loss: 0.12, Validation Accuracy: 98.04%
Epoch [3/50], Step [100/100], Loss: 0.09608691440196708, Acc : 0.9635000000000007
Epoch [3/50], Train Loss: 0.008711100555956364, Train Accuracy: 96.35%, Validation Loss: 0.18, Validation Accuracy: 98.04%
Epoch [4/50], Step [100/100], Loss: 0.10309693551156669, Acc : 0.9615000000000008
Epoch [4/50], Train Loss: 0.025748586282134056, Train Accuracy: 96.15%, Validation Loss: 0.13, Validation Accuracy: 98.04%
Epoch [5/50], Step [100/100], Loss: 0.10727794704958797, Acc : 0.957500000000001
Epoch [5/50], Train Loss: 0.08305330574512482, Train Accuracy: 95.75%, Validation Loss: 0.21, Validat

KeyboardInterrupt: 

In [ ]:


step = np.linspace(0, 100, 10500)

fig, ax = plt.subplots(figsize=(8,5))
plt.plot(step, np.array(loss_values))
plt.title("Step-wise Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()



In [ ]:

plt.plot(epoch_nums, training_acc, 'bo', label = 'Training acc')
plt.plot(epoch_nums, validation_acc, 'r', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epoch_nums, training_loss, 'bo', label = 'Training loss')
plt.plot(epoch_nums, validation_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [75]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.BCELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0001)

def train(train_loader, model, criterion, optimizer, steps):
    #model.train()
    running_loss = 0
    total = 0
    correct = 0
    accuracy = 0.0
    for i, (images, labels) in enumerate(islice(train_loader, steps)):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        _, preds = torch.max(outputs.data, 1)
        #labels = labels.unsqueeze(1)
        loss = criterion(outputs, labels.float().view(-1,1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        #acc += torch.round(outuput = label.flow().view())/len(labels)
        total += labels.size(0)
        correct += (preds == labels).sum().item()
    accuracy = correct / total
    return running_loss / steps, accuracy 


def validate(validation_loader, model, criterion, steps):
    #model.eval()
    running_loss = 0
    total = 0
    correct = 0
    accuracy = 0.0
    with torch.no_grad():
        for i, (images, labels) in enumerate(islice(validation_loader, steps)):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs.data, 1)
            #labels = labels.unsqueeze(1)
            loss = criterion(outputs, labels.float().view(-1,1))
            running_loss += loss.item()
            
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    accuracy = correct / total
    return running_loss / steps, accuracy


def train_and_evaluate(train_loader, validation_loader, model, criterion, optimizer, epochs, train_steps, val_steps):
    metrics = {'train_acc': [], 'val_acc': [], 'train_loss': [], 'val_loss': []}
    for epoch in range(epochs):
        # train and get train accuracy
        train_loss, train_acc = train(train_loader, model, criterion, optimizer, train_steps)
        metrics['train_acc'].append(train_acc)
        metrics['train_loss'].append(train_loss)

        # validate and get validation accuracy
        val_loss, val_acc = validate(validation_loader, model, criterion, val_steps)
        metrics['val_acc'].append(val_acc)
        metrics['val_loss'].append(val_loss)

        print(f"Epoch: {epoch+1}, Train Loss: {train_loss}, Train Accuracy: {train_acc}, Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")
        
    return metrics

metrics = train_and_evaluate(train_loader, val_loader, model, criterion, optimizer, 50, 100, 50)


KeyboardInterrupt: 